In [19]:
import pandas as pd
import requests

In [20]:
from retrieve_post_by_userid import RetrievePostByUserId
r = RetrievePostByUserId(sessionid="")

In [21]:

import re
def get_threads_userpk(username:str):
    
    headers = {
        "Accept": "*/*",
        "Accept-Language": "it-IT,it;q=0.8",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://www.threads.net",
        "Priority": "u=1, i",
        "Referer": f"https://www.threads.net/@{username}",
        "Sec-CH-UA": '"Not(A:Brand";v="99", "Brave";v="133", "Chromium";v="133"',
        "Sec-CH-UA-Full-Version-List": '"Not(A:Brand";v="99.0.0.0", "Brave";v="133.0.0.0", "Chromium";v="133.0.0.0"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Model": "",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-CH-UA-Platform-Version": '"19.0.0"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-ASBD-ID": "129477",
        "X-FB-LSD": "AVrgJ17PSUU"
    }
    # cookies = {
    #         "sessionid": sessionid
    # }
    data = {
        "client_previous_actor_id": "",
        "route_url": f"/@{username}",
        "routing_namespace": "barcelona_web",
        "__user": "0",
        "__a": "1",
        "__req": "m",
        "__hs": "20128.HYP:barcelona_web_pkg.2.1...0",
        "dpr": "1",
        "__comet_req": "29",
        "lsd": "AVrgJ17PSUU",
    }
    response = requests.post(
            url = "https://www.threads.net/ajax/navigation/",
            headers=headers,
            data = data
                # cookies=cookies
            )
    try:
        if "viewer does not follow private user" in response.text:
            return None
        text = response.text
        user_id_key_value = re.search('"user_id":"(\\d+)",', text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

id = get_threads_userpk("mlssfshn")
print(id)

187264532


In [22]:
df = pd.read_csv("followersAI.csv")
df

,thread_user_pk,thread_follower_pk,thread_username,thread_follower_username
0,6.331078e+10,6.408954e+10,mlssfshn,gabriel_lagon
1,6.338794e+10,6.965537e+10,thisisaltgian,beiyo_
2,6.338794e+10,6.313327e+10,thisisaltgian,realwanita
3,6.338794e+10,6.322924e+10,thisisaltgian,pelumi__i
4,6.338794e+10,6.342134e+10,thisisaltgian,olufisola_
...,...,...,...,...
25182,6.329816e+10,6.322772e+10,msmarshellw,corinnespartana
25183,6.798282e+10,6.322772e+10,affyayomikun,corinnespartana
25184,6.825805e+10,6.322772e+10,quentin.d.h,corinnespartana
25185,6.325159e+10,6.322772e+10,ohnelleyeah,corinnespartana


In [23]:
def analize_post(data_dict,thread_user_pk):
    extracted_posts = []
    for edge in data_dict.get("data", {}).get("mediaData", {}).get("edges", []):
        node = edge.get("node", {})
        for item in node.get("thread_items", []):
            post = item.get("post", {})
            extracted = {
                "id": post.get("id"),
                "post_pk": post.get("pk"),
                "caption_text": post.get("caption", {}).get("text") if post.get("caption") else None,
                "like_count": post.get("like_count"),
                "taken_at": post.get("taken_at"),
                "username": post.get("user", {}).get("username"),
                "user_pk": post.get("user", {}).get("pk"),
                "quote_count": post.get("text_post_app_info", {}).get("quote_count"),
                "repost_count": post.get("text_post_app_info", {}).get("repost_count"),
                "reshare_count": post.get("text_post_app_info", {}).get("reshare_count"),
                "thread_user_pk": thread_user_pk,
            }
            extracted_posts.append(extracted)
            
    return extracted_posts


In [24]:
dfpost = pd.read_csv("postAI.csv")
dfpost
user_processed = dfpost["thread_user_pk"].astype(str).unique()
len(user_processed)

4586

In [30]:
user_processed =list(user_processed)
user_processed.append("69153761497.0")
user_processed.append("63069579229.0")
user_processed.append("64147865290.0")
user_processed.append("68791764906.0")
user_processed.append("63456322225.0")
user_processed.append("64871352972.0")




In [ ]:
previus = ""
for idx ,row in df.iterrows():
    if idx < 4163 :
        continue
    if str(row["thread_follower_username"]) == previus:
        continue
    if not str(row["thread_follower_pk"]) in user_processed:
        previus = row["thread_follower_username"]
        print(row["thread_follower_pk"], " ",row["thread_follower_username"])
        try:
            id = get_threads_userpk(row["thread_follower_username"])
        except:
            if "46962366031" == str(get_threads_userpk("shawnecesellsatl")):
                user_processed.append(row["thread_follower_pk"])
                continue
        posts = r.retrieve_thread_by_userid(userid=str(id))

        if not id or not posts["data"]["mediaData"]["edges"]:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_follower_username"],
                "user_pk": id if id else "",
                "thread_follower_pk": row["thread_follower_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None
            }])
        else:
            temp_df = pd.DataFrame(analize_post(posts,row["thread_follower_pk"]))
        user_processed.append(str(row["thread_follower_pk"]))
        temp_df.to_csv("postAI.csv", mode='a', header=False, index=False)
        print("Processed ", idx, "on ", len(df))

70098630844.0   inthegranite617_ttdl_tpd
70872131267.0   ehsannoroozi732
8157073487.0   si.mone5257
Processed  3698 on  25187
66784100180.0   mdbelalchowdury
Processed  3699 on  25187
59310770036.0   hassaneinakchi
Processed  3700 on  25187
70138943853.0   ssemandambabaalifaisal
Processed  3701 on  25187
50126802411.0   sandaaye3
Processed  3702 on  25187
63146009406.0   carolstone851
Processed  3703 on  25187
64170401702.0   marioespolon
Processed  3704 on  25187
7163188820.0   shantikumarrao
Processed  3705 on  25187
45850705372.0   joshy6255
Processed  3706 on  25187
59845327418.0   amri.abdelhafid
Processed  3707 on  25187
5793801480.0   mickybungle
Processed  3708 on  25187
307327157.0   pornchaiptt
Processed  3709 on  25187
8715112523.0   moridepiremoqanam
Processed  3710 on  25187
2174723023.0   neiagomeslima
Processed  3711 on  25187
3516163929.0   ellabay1966
Processed  3712 on  25187
30576625344.0   kleberdos53
Processed  3713 on  25187
8470640957.0   rosalvoferraz.news
Proce

KeyboardInterrupt: 

In [ ]:
dfpost

,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count,thread_user_pk
0,3558787428782179764_187264532,3.558787e+18,Behind the scenes,0.0,1.738461e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
1,3557970465522674878_187264532,3.557970e+18,A little over 24 hours left to vote.\n\nhttps:...,0.0,1.738363e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
2,3557782854629238181_187264532,3.557783e+18,"Galaxy Quest \nLots of cutting holes, playing ...",1.0,1.738341e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
3,3557776470965660229_187264532,3.557776e+18,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.0,1.738340e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
4,3556000079513982405_187264532,3.556000e+18,Over due to post videos of my work for @salina...,0.0,1.738128e+09,mlssfshn,187264532,0.0,0.0,NaN,6.331078e+10
...,...,...,...,...,...,...,...,...,...,...,...
89801,3536005312478769473_7189018534,3.536005e+18,La Familia que ha Mi Señor le plació prestarm...,0.0,1.735745e+09,thebeauty071963,7189018534,0.0,0.0,NaN,6.871577e+10
89802,3535623203146758796_7189018534,3.535623e+18,"Feliz Nuevo Año ♥️que se avecina, Gracias Padr...",0.0,1.735699e+09,thebeauty071963,7189018534,0.0,0.0,NaN,6.871577e+10
89803,3535260610037256253_7189018534,3.535261e+18,"Me encantó, muy cierto ☺️",0.0,1.735656e+09,thebeauty071963,7189018534,0.0,0.0,NaN,6.871577e+10
89804,3535244455717268490_7189018534,3.535244e+18,AMEN Y AMEN 🙏🏻🥰,0.0,1.735654e+09,thebeauty071963,7189018534,0.0,0.0,NaN,6.871577e+10
